## 问题1：[最长连续序列](https://leetcode.cn/problems/longest-consecutive-sequence/description/)
### 题目
给定一个未排序的整数数组nums, 找出数字连续的最长序列(不要求序列元素在原数组中连续)的长度。

请你设计并实现时间复杂度为 O(n) 的算法解决此问题。

### 分析
- 由于需要时间复杂度为n的算法, 所以我们不能排序. 为解决序列问题需要, 明确每一个数时候能对最终的答案有贡献, 使用哈希表来记录每一个数字所在的连续序列长度。比如，如果存在2,3,4的连续序列, 则哈希表中2,3,4对应的值即为3(表示这些值所在连续序列的长度)
- 遍历nums, 当出现i时, 判断i是否已经被使用过了(已经在哈希表中), 如果是则跳过。如果不是则需要加入到hash表中。
    - 对于新加入的值我们需要考察是否该值会使得某两个区间合并，比如已经存在[1，3]区间和[5,6]区间，如果加入4，则可以将连续区间扩大为[1，6]
    - 因此我们只需要考察哈希表中3，5的值，将hash(3) + hash(5) + 1 即为合并后的长度
    - 注意: 我们此时需要更新区间[1,6]中每一个对应的hash值吗？其实不用我们发现我们只用更新端点的值即可(只用更新1,6对应的值), 因为2，3，4，5这些值在下一次碰见时是没有任何作用的

In [ ]:
def process(nums):
    hash_dict = {}
    ans = 0
    for i in nums:
        if i not in hash_dict:
            left_inter = hash_dict.get(i-1, 0)
            right_inter = hash_dict.get(i+1, 0)
            current_inter = left_inter + right_inter + 1
            # 更新最大值
            ans = max(ans, current_inter)
            # 更新端点值
            hash_dict[i] = current_inter
            hash_dict[i - left_inter] = current_inter
            hash_dict[i + right_inter] = current_inter
            print(hash_dict)
    return ans

nums = [100,4,200,1,3,2]
process(nums)

## 问题2：[字母异位词分组](https://leetcode.cn/problems/group-anagrams/description/)
### 题目
给你一个字符串数组，请你将 字母异位词 组合在一起。可以按任意顺序返回结果列表。

字母异位词: 是由重新排列源单词的所有字母得到的一个新单词。
### 分析
- 使用一种特殊的哈希方法, 将每个元素映射成为一个则26位的字符串: ord(字母)-97表示位置, 如果出现了1次则对应位置上+1, 最后返回一个字符串，表示这个单词的特征, 最后可以使用tuple([1,2,3])的方式实现作为键.

In [ ]:
from collections import defaultdict

def hash_word(word):
    ans = [0] * 26
    for i in word:
        ans[ord(i)-97] += 1
    return str(ans)

def process(strs):
    hash_map = defaultdict(list)
    for word in strs:
        hash_map[hash_word(word)].append(word)
    
    return [value for _, value in hash_map.items()]

strs = ["bdddddddddd","bbbbbbbbbbc"]
process(strs)
hash_word("bdddddddddd")

## 问题3：[移动零](https://leetcode.cn/problems/move-zeroes/description/)
### 题目
给定一个数组 nums，编写一个函数将所有 0 移动到数组的末尾，同时保持非零元素的相对顺序。
请注意 ，必须在不复制数组的情况下原地对数组进行操作。

### 分析
- 遍历时采用remove(0)（每次只会去除第一个数字） + append(0)将0移动到末尾
- **注意**: 不可以在遍历nums对nums进行修改！！因此我们需要先遍历完成记录有多少个0
```python
a = [1,2,3,4]
iterator = iter(a)
print(next(iterator)) # 此时游标来到了index=0
a.pop(0)              # 由于删除了0位置, 因此此时的a为[2,3,4]
print(next(iterator)) # 此时游标来到了index=1, 因此打印的时[2，3，4]的index=1的位置即为3
```

### 优化
- 从左往右遍历时, 如果去修改nums即立刻pop(index) + append(0), 会导致index偏移，由于我们删除后会添加，因此我们从右往左遍历，删除和添加后index向前移动不会出现跳过某些数字的情况


In [ ]:
def process(nums):
    count = 0
    for num in nums:
        if num == 0:
            count += 1
    for _ in range(count):
        nums.remove(0)
        nums.append(0)
        
    return nums

# 优化
def process(nums):
    right = len(nums)-1
    while right >= 0:
        if nums[right] == 0:
            nums.pop(right)
            nums.append(0)
        right -= 1

## 问题4：[盛最多水的容器](https://leetcode.cn/problems/container-with-most-water/description/)
### 题目
给定一个长度为n的整数数组height。有n条垂线，第i条线的两个端点是(i, 0)和(i, height[i]) 。
找出其中的两条线(作为墙壁), 使得它们与x轴共同构成的容器可以容纳最多的水。返回容器可以储存的最大水量。

### 分析
- 与接雨水不同的是, 这里需要放弃移动后的水量
- 我们使用左右指针的方式遍历整个情况，水量 = min(左指针, 右指针) * (左右指针间距)
- 这里我们移动指针的原则是: 移动左右指针所指位置较小的那个值(相等移动任意一个都可以)。 具体证明见[参考](https://leetcode.cn/problems/container-with-most-water/solutions/207215/sheng-zui-duo-shui-de-rong-qi-by-leetcode-solution/?envType=study-plan-v2&envId=top-100-liked)
    - 如果移动大数值的指针, 则一定有min(新区间) <= min(老区间), 新区间长度 <= 老区间长度, 因此移动大数值指针一定不会更优
- 双指针遍历完后, 得到的最大值就是我们需要的

In [ ]:
def process(height):
    left = 0
    right = len(height)-1
    ans = 0
    while left <= right:
        ans = max(ans, min(height[left], height[right]) * (right - left))
        if height[left] <= height[right]:
            left += 1
        else:
            right -= 1
    return ans

height = [1,8,6,2,5,4,8,3,7]
process(height)

## 问题5：[三数之和](https://leetcode.cn/problems/3sum/description/)
### 题目
给你一个整数数组nums，判断是否存在三元组[nums[i], nums[j], nums[k]] 满足 i != j、i != k 且 j != k ，同时还满足 nums[i] + nums[j] + nums[k] == 0 。请

你返回所有和为 0 且不重复的三元组。

注意：答案中不可以包含重复的三元组。

### 分析
- (对于无序数组) 暴力解法使用三层循环遍历, 时间复杂度为O(n^3)
- 可以使用排序+双指针的方式(类似于两数之和), 由于需要不重复的三元组, 因此在最外层排序时如果发现当前数值与前一个相同, 则可以直接跳过。
- 当在内层发现了一组合适的值时, 需要继续判断, 因为可能存在 left+1和right-1后仍能满足目标和的情况。但为了减少遍历, 可以将left和right同时移动到与当前值不同的地方。

In [9]:
def process(nums, target):
    ans = set()
    for index_1, num_1 in enumerate(nums):
        for index_2, num_2 in enumerate(nums[index_1+1:]):  # 从第一个元素之后开始
            # 将第三层循环的判断交给in
            if target - num_1 - num_2 in nums[index_2 + index_1+2:]:
                ans.add(tuple(sorted([num_1, num_2,target - num_1 - num_2])))
        
    return [list(i) for i in ans]

nums = [-1,0,1,2,-1,-4]
process(nums, 0)

[[-1, 0, 1], [-1, -1, 2]]

In [10]:
def process(nums, target):
    ans = []
    nums.sort()
    n = len(nums)
    for index, num in enumerate(nums):
        # 当前数字已经用过了则可以跳过
        if index != 0 and nums[index] == nums[index-1]:
            continue
        left = index + 1
        right = n - 1
        while left < right:  
            if nums[left] + nums[right] + num > target:
                right -= 1
            elif nums[left] + nums[right] + num < target:
                left += 1
            else:
                ans.append([num, nums[left], nums[right]])  # 找到一组符合要求的数据了
                # 需要继续遍历
                while left < right and nums[left] == nums[left+1]:
                    left += 1
                while left < right and nums[right] == nums[right-1]:
                    right -= 1
                    
                # 来到这里时, 目前left和right都仍为num, nums[left], nums[right], 主动向前移动
                left += 1 
                right -= 1
    return ans

nums = [-1,0,1,2,-1,-4]
process(nums, 0)

[[-1, -1, 2], [-1, 0, 1]]

In [ ]:
s = set()
s.add(tuple(sorted([1,1,2])))
s.add(tuple(sorted([2,1,1])))
s

## 问题6：[接雨水](https://leetcode.cn/problems/trapping-rain-water/)
### 题目
给定 n 个非负整数表示每个宽度为 1 的柱子的高度图height，计算按此排列的柱子，下雨之后能接多少雨水。

### 分析
- 先对高度进行倒序遍历得到, 右侧最大数组right_max, right_max[i]表示height[i:]中最大的值
- 然后在顺序遍历, 遍历过程中记录left_max, 表示height[:i]的最大值, 当遍历到i时候, 当前单元格能接的雨水为 min(left_max, right_max[i]) - height[i]
- 边界条件都为0

## 优化
- 我们可以只使用一次遍历即可完成, 双指针初始位置l,r分别指向0和len(height)-1。
- 循环l < r, 并维护right_max, 和left_max。
    - left_max更小, 则当前l所对应能接的雨水就是left_max - height[l], 因为虽然right_max没有遍历完，但由于right_max是单调递增的所以之后的right_max一定比left_max更大, 所以l位置对应所能接的雨水也为left_max - height[l]
    - 同理，如果right_max更小, 则当前r所对应能接的雨水就是right_max - height[r]

In [5]:
def process(height):
    # 倒序遍历得到右侧最大数组
    right_max_list = []
    right_max = 0 
    for num in height[::-1]:
        right_max = max(right_max, num)
        right_max_list.insert(0, right_max)
    
    ans = 0
    left_max = 0
    for i, num in enumerate(height):
        left_max = max(left_max, num)
        ans += (min(left_max, right_max_list[i]) - num)

    return ans
    

height = [4,2,0,3,2,5]
process(height)


9

In [ ]:
def process(height):
    l, r = 0, len(height)-1
    left_max, right_max = 0, 0
    ans = 0

    while l < r:
        left_max = max(height[l], left_max)
        right_max = max(height[r], right_max)
        if left_max < right_max:
            ans += left_max - height[l]
            l += 1  # 左指针往前移动
        else:
            ans += right_max - height[r]
            r -= 1
    # 退出时 l==r, 并且一定是指向的最大值, 因此不用最后再计算一次
    # 当然循环时直接 l <= r也是可以的
    return ans

## 问题7：[和为 K 的子数组](https://leetcode.cn/problems/subarray-sum-equals-k/description/)
### 题目
给你一个整数数组nums和一个整数k ，请你统计并返回该数组中和为k的**子数组**(连续非空数组)的个数。注意nums是无序的, 并且存在负数

### 分析
- 由于nums无序且存在负数, 因为不能使用滑动窗口(因为不能保证窗口移动的一致性)
- 因此本题采用前缀和的算法, s[i]表示从[0 : i]的和, 那么[i : j]的和为 s[j] - s[i-1]. 我们遍历s[i]时, 将具有相同前缀和的个数记录在哈希表count中. 在遍历j时，判断是否存在count[ s[j] - k ] , 如果存在则直接ans += count[ s[j] - k ]
- 注意我们初始化count时必须把 0 放进去, 也就是count[0] = 1, 这样才可以保证存在s[0] - 0 = k的情况  

In [4]:
from collections import defaultdict

def process(nums, k):
    ans = 0
    n = len(nums)
    s = [0] * n
    s[0] = nums[0]
    for i in range(1, n):
        s[i] += s[i-1] + nums[i]

    
    count = defaultdict(int)
    count[0] = 1
    for j in range(n):
        ans += count[s[j] - k]
        count[s[j]] += 1
            
    return ans

nums = [1,1,1]
k = 2
process(nums, k)

2

## 问题8：[环形链表 II](https://leetcode.cn/problems/linked-list-cycle-ii/description/)
### 题目
给定一个链表的头节点head，返回链表开始入环的第一个节点。 如果链表无环，则返回 null。


### 分析
- 使用双指针，分别命名为slow和fast，起始位置在链表的开头。每次fast前进两步，slow前进一步。
    - 如果fast可以走到尽头，那么说明没有环路；
    - 如果 fast 可以无限走下去，那么说明一定有环路，且一定存在一个时刻slow和fast相遇。如果在fast到达终点前两者相遇了则说明一定有环
- 更进一步的: 当 slow 和 fast 第一次相遇时，我们将 fast 重新移动到链表开头，并让 slow 和 fast 每次都前进一步。当 slow 和 fast 第二次相遇时，相遇的节点即为环路的开始点。

In [ ]:
def process(head):
    slow = head
    fast = head
    while fast != None and fast.next != None:
        fast = fast.next.next
        slow = slow.next

        if fast == slow:  # 如果两者相遇了, 则说明存在环
            fast = head
            while fast != slow: # 一直遍历直到两者相遇
                fast = fast.next
                slow = slow.next
            return slow
    return


## 问题9：[分割等和子集](https://leetcode.cn/problems/partition-equal-subset-sum/)
### 题目
给你一只包含正整数的非空数组nums 。请你判断是否可以将这个数组分割成两个子集，使得两个子集的元素和相等。

### 分析
- 先计算nums数组和，如果为奇数则直接返回false
- 将和//2即可得到目标和target, 题目转化为在数组中寻找和为target的子集。
- 我们可以使用0-1背包算法(动态规划), 题目"正整数"也是在暗示我们. dp[i][j]表示截至到i位置, 能否有子集为和为j
- 转移方程: dp[i][j] = dp[i-1][j] or dp[i-1][j-nums[i]]
- 初始条件dp[i][0] = true

### 优化
- 空间优化, 因为dp[i][j]只依赖于dp[i-1]因此我们可以使用一维数组堆空间进行优化
- 注意为了保证j-nums[i]为i-1的值, 我们需要逆序遍历


In [3]:
def process(nums):
    target = sum(nums)
    if target % 2 == 1:
        return False
    target //= 2

    n = len(nums)
    dp = [[False] * (target+1) for _ in range(n)]
    # 初始化条件
    for i in range(n):
        dp[i][0] = True

    for i, num in enumerate(nums):
        for j in range(target+1):
            if j < num:
                dp[i][j] = dp[i-1][j]
            else:
                dp[i][j] = dp[i-1][j] or dp[i-1][j-num]

    return dp[n-1][target]

nums = [1,2,3,5]
process(nums)


False

In [ ]:
# 空间优化版
def process(nums):
    target = sum(nums)
    if target % 2 == 1:
        return False
    target //= 2

    n = len(nums)
    dp = [False] * (target+1)
    # 初始化条件
    dp[0] = True

    for i, num in enumerate(nums):
        for j in range(target, num-1, -1):  # 只用遍历到num即可
            dp[j] = dp[j] or dp[j-num]

    return dp[target]

## 问题10：[路径总和 III](https://leetcode.cn/problems/path-sum-iii/)
### 题目
给定一个二叉树的根节点root，和一个整数targetSum，求该二叉树里节点值之和(节点数值可能为负数)等于targetSum的路径的数目。

注意: 路径不需要从根节点开始，也不需要在叶子节点结束，但是路径方向必须是向下的（只能从父节点到子节点）。

### 分析
- 与【问题7：和为 K 的子数组】一样由于存在节点为负数的情况, 我们不能使用递归(不能知道终止条件)
- 我们应该应该采用前缀和+哈希表的方式: 在遍历二叉树时, s表示从根节点到当前节点的节点和(由于遍历二叉树需要使用dfs, 因此在使用时包含这个s记录下根节点到此处的节点和), 我们在遍历二叉树时, 将具有相同前缀和(根节点到此节点的路径和)的个数记录在哈希表count中. 在遍历到节点j时，判断是否存在count[s - targetSum ] , 如果存在则直接ans += count[ s - targetSum ]

In [ ]:
from collections import defaultdict

def process(root, targetSum):
    ans = 0
    count = defaultdict(int)
    count[0] = 1   # 前缀和为0的有1个

    def dfs(node, path_sum):  # 根节点到当前节点的节点和(未计数当前节点)
        if node == None:
            return
        nonlocal ans
        path_sum += node.val
        ans += count[path_sum - targetSum]
        count[path_sum] += 1
        # 分别递归左右子树
        dfs(node.left, path_sum)
        dfs(node.right, path_sum)

        # 注意还原现场(因为要去遍历其他非当前子树的节点了, 这个值应该还原)
        count[path_sum] -= 1
    # 初始遍历
    dfs(root, 0)
    return ans



## 问题11：[]()
### 题目


### 分析